# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 16 2023 11:14AM,255096,22,638460,"NBTY Global, Inc.",Released
1,Jan 16 2023 11:11AM,255094,19,MSP217559,"Methapharm, Inc.",Released
2,Jan 16 2023 11:10AM,255093,10,9014378-BO,Eywa Pharma Inc.,Released
3,Jan 16 2023 11:10AM,255093,10,9027591,Eywa Pharma Inc.,Released
4,Jan 16 2023 11:10AM,255093,10,9038451,Eywa Pharma Inc.,Released
5,Jan 16 2023 11:10AM,255093,10,9038456,Eywa Pharma Inc.,Released
6,Jan 16 2023 11:10AM,255093,10,9038457,Eywa Pharma Inc.,Released
7,Jan 16 2023 10:56AM,255090,19,ADV80010422,"AdvaGen Pharma, Ltd",Released
8,Jan 16 2023 10:56AM,255090,19,ADV80010423,"AdvaGen Pharma, Ltd",Released
9,Jan 16 2023 10:56AM,255090,19,ADV80010424,"AdvaGen Pharma, Ltd",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
20,255091,Released,14
21,255092,Released,1
22,255093,Released,5
23,255094,Released,1
24,255096,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
255091,NaN,14.0
255092,NaN,1.0
255093,NaN,5.0
255094,NaN,1.0
255096,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
255006,40.0,12.0
255049,0.0,42.0
255050,0.0,16.0
255053,0.0,1.0
255054,7.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
255006,40,12
255049,0,42
255050,0,16
255053,0,1
255054,7,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,255006,40,12
1,255049,0,42
2,255050,0,16
3,255053,0,1
4,255054,7,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,255006,40,12
1,255049,,42
2,255050,,16
3,255053,,1
4,255054,7,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 16 2023 11:14AM,255096,22,"NBTY Global, Inc."
1,Jan 16 2023 11:11AM,255094,19,"Methapharm, Inc."
2,Jan 16 2023 11:10AM,255093,10,Eywa Pharma Inc.
7,Jan 16 2023 10:56AM,255090,19,"AdvaGen Pharma, Ltd"
57,Jan 16 2023 10:55AM,255092,22,FUJIFILM Diosynth Biotechnologies Texas LLC
58,Jan 16 2023 10:54AM,255091,10,Bio-PRF
72,Jan 16 2023 10:45AM,255089,10,"Methapharm, Inc."
83,Jan 16 2023 10:42AM,255082,10,ISDIN Corporation
138,Jan 16 2023 10:40AM,255088,19,"GUSA Granules USA, Inc."
139,Jan 16 2023 10:39AM,255087,21,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jan 16 2023 11:14AM,255096,22,"NBTY Global, Inc.",,1
1,Jan 16 2023 11:11AM,255094,19,"Methapharm, Inc.",,1
2,Jan 16 2023 11:10AM,255093,10,Eywa Pharma Inc.,,5
3,Jan 16 2023 10:56AM,255090,19,"AdvaGen Pharma, Ltd",,50
4,Jan 16 2023 10:55AM,255092,22,FUJIFILM Diosynth Biotechnologies Texas LLC,,1
5,Jan 16 2023 10:54AM,255091,10,Bio-PRF,,14
6,Jan 16 2023 10:45AM,255089,10,"Methapharm, Inc.",,11
7,Jan 16 2023 10:42AM,255082,10,ISDIN Corporation,,55
8,Jan 16 2023 10:40AM,255088,19,"GUSA Granules USA, Inc.",,1
9,Jan 16 2023 10:39AM,255087,21,"NBTY Global, Inc.",1,


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jan 16 2023 11:14AM,255096,22,"NBTY Global, Inc.",1,
1,Jan 16 2023 11:11AM,255094,19,"Methapharm, Inc.",1,
2,Jan 16 2023 11:10AM,255093,10,Eywa Pharma Inc.,5,
3,Jan 16 2023 10:56AM,255090,19,"AdvaGen Pharma, Ltd",50,
4,Jan 16 2023 10:55AM,255092,22,FUJIFILM Diosynth Biotechnologies Texas LLC,1,
5,Jan 16 2023 10:54AM,255091,10,Bio-PRF,14,
6,Jan 16 2023 10:45AM,255089,10,"Methapharm, Inc.",11,
7,Jan 16 2023 10:42AM,255082,10,ISDIN Corporation,55,
8,Jan 16 2023 10:40AM,255088,19,"GUSA Granules USA, Inc.",1,
9,Jan 16 2023 10:39AM,255087,21,"NBTY Global, Inc.",,1


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jan 16 2023 11:14AM,255096,22,"NBTY Global, Inc.",1,
1,Jan 16 2023 11:11AM,255094,19,"Methapharm, Inc.",1,
2,Jan 16 2023 11:10AM,255093,10,Eywa Pharma Inc.,5,
3,Jan 16 2023 10:56AM,255090,19,"AdvaGen Pharma, Ltd",50,
4,Jan 16 2023 10:55AM,255092,22,FUJIFILM Diosynth Biotechnologies Texas LLC,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jan 16 2023 11:14AM,255096,22,"NBTY Global, Inc.",1.0,NaN
1,Jan 16 2023 11:11AM,255094,19,"Methapharm, Inc.",1.0,NaN
2,Jan 16 2023 11:10AM,255093,10,Eywa Pharma Inc.,5.0,NaN
3,Jan 16 2023 10:56AM,255090,19,"AdvaGen Pharma, Ltd",50.0,NaN
4,Jan 16 2023 10:55AM,255092,22,FUJIFILM Diosynth Biotechnologies Texas LLC,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jan 16 2023 11:14AM,255096,22,"NBTY Global, Inc.",1.0,0.0
1,Jan 16 2023 11:11AM,255094,19,"Methapharm, Inc.",1.0,0.0
2,Jan 16 2023 11:10AM,255093,10,Eywa Pharma Inc.,5.0,0.0
3,Jan 16 2023 10:56AM,255090,19,"AdvaGen Pharma, Ltd",50.0,0.0
4,Jan 16 2023 10:55AM,255092,22,FUJIFILM Diosynth Biotechnologies Texas LLC,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2550785,210.0,7.0
12,765178,21.0,1.0
15,765105,70.0,40.0
19,765272,52.0,0.0
20,255070,0.0,1.0
21,255087,0.0,1.0
22,765241,3.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2550785,210.0,7.0
1,12,765178,21.0,1.0
2,15,765105,70.0,40.0
3,19,765272,52.0,0.0
4,20,255070,0.0,1.0
5,21,255087,0.0,1.0
6,22,765241,3.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,210.0,7.0
1,12,21.0,1.0
2,15,70.0,40.0
3,19,52.0,0.0
4,20,0.0,1.0
5,21,0.0,1.0
6,22,3.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,210.0
1,12,Released,21.0
2,15,Released,70.0
3,19,Released,52.0
4,20,Released,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,19,20,21,22
Status,,,,,,,
Executing,7.0,1.0,40.0,0.0,1.0,1.0,0.0
Released,210.0,21.0,70.0,52.0,0.0,0.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,19,20,21,22
0,Executing,7.0,1.0,40.0,0.0,1.0,1.0,0.0
1,Released,210.0,21.0,70.0,52.0,0.0,0.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,19,20,21,22
0,Executing,7.0,1.0,40.0,0.0,1.0,1.0,0.0
1,Released,210.0,21.0,70.0,52.0,0.0,0.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()